In [1]:
def jaccard_similarity(set1, set2):
    set1_upper = {element.upper() for element in set1}
    set2_upper = {element.upper() for element in set2}

    intersection = len(set1_upper.intersection(set2_upper))
    union = len(set1_upper.union(set2_upper))
    
    return intersection / union if union != 0 else 0

In [3]:
sim = jaccard_similarity('dog', 'dogo')

In [1]:
def find_trigrams(text: str) -> set:
    """
    returns a set of ngrams for the given string
    :param text: the string to find ngrams for
    :param number: the length the ngrams should be. defaults to 3 (trigrams)
    :return: set of ngram strings
    """

    if not text:
        return set()

    words = [f'  {x} ' for x in re.split(r'\W+', text.lower()) if x.strip()]

    ngrams = set()

    for word in words:
        for x in range(0, len(word) - 4):
            ngrams.add(word[x:x+3])

    return ngrams

In [18]:
import nltk
nltk.download('punkt')

def tokenize(text):
    if isinstance(text, str):  
        return set(nltk.word_tokenize(text.upper()))
    else:
        return set()
    
list1 = tokenize('Matt Böhm')
list2 = tokenize('Matt Boehm')
print(list1)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'MATT', 'BÖHM'}


In [16]:
def jaccard_similarity_case_insensitive(list1, list2):
    s1 = set(map(str.upper, list1))
    s2 = set(map(str.upper, list2))
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))


# Example usage:
list1 = ['Matt', 'Böhm']
list2 = ['Matt', 'Boehm']

similarity = jaccard_similarity_case_insensitive(list1, list2)
print(similarity)


0.3333333333333333


In [15]:
sim = jaccard_similarity_case_insensitive('DOG', 'dog')
print(sim)

1.0
